# Libraries

In [1]:
import datetime
import json
import pandas as pd
import requests
import time
import random
import sys
import os
import os.path

from mcmetadata import extract
# pip install mediacloud-metadata

from tqdm.notebook import trange  # to display a progress bar
import ipywidgets as widgets
from IPython.display import display

# Init the connection
# mc = mediacloud.api.DirectoryApi("56196a395ee77c33a296073fa08e72f541362a10")

# Dataset

In [2]:
# query = "Query_NYT_5y_surveillance"
# query = "Query_NYT_13y_surveillance"
# query = "Query_US_5y_Svalbard"
query = "Query_AI-magazines-mediacloud-2021-24"

CSV_FILE = query + ".csv"
FEATHER_FILE = query + ".feather"

if os.path.isfile(FEATHER_FILE):
    df = pd.read_feather(FEATHER_FILE)

else:
    df = pd.read_csv(CSV_FILE)

    df["year"] = df.loc[:, "publish_date"]
    df["year"] = df["year"].str[:4]
    df["text"] = ""
    df["text_downloaded"] = False

    # df = df.drop(
    #     [
    #         "ap_syndicated",
    #         "collect_date",
    #         "feeds",
    #         "guid",
    #         "media_id",
    #         "media_name",
    #         "media_url",
    #         "metadata",
    #         "language",
    #         "processed_stories_id",
    #         "publish_date",
    #         "story_tags",
    #         "word_count",
    #     ],
    #     axis=1,
    # )

df.sample(1)

,id,indexed_date,language,media_name,media_url,publish_date,title,url,year,text,text_downloaded
521,6317fa629d2157ed616226095f66e8fb791a9b2a800425...,2024-03-14 06:12:57.462522,en,technologyreview.com,technologyreview.com,2024-03-12,Why we need better defenses against VR cyberat...,https://www.technologyreview.com/2024/03/12/10...,2024,,False


In [3]:
# Error Counter
total = widgets.IntText(description="Total", value=df.shape[0])
done = widgets.IntText(description="Done", value=0)
unavailable = widgets.IntText(description="Unavailable", value=0)
yes = widgets.IntText(description="Downloaded", value=0)
no = widgets.IntText(description="Errors", value=0)
display(total, done, unavailable, yes, no)

for index in trange(0, df.shape[0]):
    if df.iloc[index].text_downloaded == True:
        done.value += 1
        continue

    API = "https://archive.org/wayback/available"
    params = {
        "url": df.iloc[index].url,
        "timeout": "300",
        "closest": "either",
        "status_code": "200",
    }
    headers = {"accept": "application/json"}

    try:
        response = requests.get(API, params=params, headers=headers)
        snapshot = response.json().get("archived_snapshots", {}).get("closest")

        if snapshot:
            metadata = extract(url=snapshot.get("url"))
            df.at[index, "text"] = metadata["text_content"]
            df.at[index, "text_downloaded"] = True
            yes.value += 1

        else:
            unavailable.value += 1

    except Exception as e:
        no.value += 1

    # time.sleep(random.uniform(1, 20)) # This is not bad to run in the night
    # time.sleep(random.uniform(.1, 1))

    df.to_feather(FEATHER_FILE)

NameError: name 'df' is not defined

In [2]:
# df.to_feather(FEATHER_FILE)
df.sample(10)

NameError: name 'df' is not defined

In [6]:
df.shape

(2633, 11)

In [10]:
df.to_csv('bd_AI_magazines.csv',index=False)